# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading

Load original dataset and preprocess the loaded data.

In [2]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
ori_data = np.load('../OUTPUT/test/samples/etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../OUTPUT/test/ddpm_fake_test_milestone_90.npy')

## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [3]:
for j in range(10):

    context_fid_score = []

    for i in range(iterations):
        context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
        context_fid_score.append(context_fid)
        print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')

    display_scores(context_fid_score)

# Seed 12345 Final Score:  0.13663267402399055 ± 0.005536055372540525
# Seed 12345 Final Score:  0.129
# Seed 12345 Final Score:  0.135

# Seed 2025 Final Score:  0.148
# Seed 2025 Final Score:  0.147
# Seed 2025 Final Score:  0.142
# Seed 2025 Final Score:  0.148

/mnt/data728/duyin/anaconda3/envs/torch201/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Iter 0:  context-fid = 0.1539521184869862 

Iter 1:  context-fid = 0.14197769441115665 

Iter 2:  context-fid = 0.1572108634809493 

Iter 3:  context-fid = 0.14124899379889222 

Iter 4:  context-fid = 0.14140027484046774 

Final Score:  0.14715798900369043 ± 0.009660341998219855
Iter 0:  context-fid = 0.13575071335584082 

Iter 1:  context-fid = 0.15314940546329336 

Iter 2:  context-fid = 0.1265744151430786 

Iter 3:  context-fid = 0.1451707964153082 

Iter 4:  context-fid = 0.15823098788380857 

Final Score:  0.14377526365226592 ± 0.01593801144268587
Iter 0:  context-fid = 0.13478861925439292 

Iter 1:  context-fid = 0.15159140812987948 

Iter 2:  context-fid = 0.15113372006685577 

Iter 3:  context-fid = 0.12039819637549547 

Iter 4:  context-fid = 0.14423213684225505 

Final Score:  0.14042881613377572 ± 0.016268893065113872
Iter 0:  context-fid = 0.1504375629148786 

Iter 1:  context-fid = 0.15606139937888347 

Iter 2:  context-fid = 0.15116564329397644 

Iter 3:  context-fid = 0.

## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [4]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [5]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.06502734570866069 

Iter 1:  cross-correlation = 0.04699022407400987 

Iter 2:  cross-correlation = 0.060597895333681207 

Iter 3:  cross-correlation = 0.05390166204833485 

Iter 4:  cross-correlation = 0.06074700349739004 

Final Score:  0.057452826132415334 ± 0.008784130428385006
